In [1]:
import tensorflow as tf
import numpy as np

from textloader import TextLoader
from tensorflow.python.ops.rnn_cell import BasicLSTMCell


# Global Variables

In [2]:
#
# -------------------------------------------
#
# Global variables

batch_size = 50
sampler_batch_size = 1
sequence_length = 50    
sampler_seq_len = 1

data_loader = TextLoader( ".", batch_size, sequence_length )

vocab_size = data_loader.vocab_size  # dimension of one-hot encodings (70)
state_dim = 128

num_layers = 2

tf.reset_default_graph()

reading text file


# Graph

In [3]:
#  inputs
#
# ==================================================================
# ==================================================================
# ==================================================================
#

# define placeholders for our inputs.  
# in_ph is assumed to be [batch_size,sequence_length]
# targ_ph is assumed to be [batch_size,sequence_length]

in_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='inputs' )
targ_ph = tf.placeholder( tf.int32, [ batch_size, sequence_length ], name='targets' )
in_onehot = tf.one_hot( in_ph, vocab_size, name="input_onehot" )

inputs = tf.split( 1, sequence_length, in_onehot )
inputs = [ tf.squeeze(input_, [1]) for input_ in inputs ]
targets = tf.split( 1, sequence_length, targ_ph )


s_inputs_ph = tf.placeholder( tf.int32, [ sampler_seq_len, ],
                          name='s_inputs' )
s_in_onehot = tf.one_hot( s_inputs_ph, vocab_size, name="s_input_onehot" )

s_inputs = tf.split( 1, sampler_seq_len, s_in_onehot )
s_inputs = [ input_ for input_ in s_inputs ]
# s_inputs = [ tf.squeeze(input_, [1]) for input_ in s_inputs ]

# at this point, inputs is a list of length sequence_length
# each element of inputs is [batch_size,vocab_size]

# targets is a list of length sequence_length
# each element of targets is a 1D vector of length batch_size

In [4]:
# ------------------------
# GRU definition


from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope as vs
# from tensorflow.python.ops.rnn_cell._SlimRNNCell import _linear

from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest



from tensorflow.python.ops.rnn_cell import RNNCell
 

    
    
    
class BasicGRUCell(RNNCell):
  """Basic LSTM recurrent network cell.
  The implementation is based on: http://arxiv.org/abs/1409.2329.
  We add forget_bias (default: 1) to the biases of the forget gate in order to
  reduce the scale of forgetting in the beginning of the training.
  It does not allow cell clipping, a projection layer, and does not
  use peep-hole connections: it is the basic baseline.
  For advanced models, please use the full LSTMCell that follows.
  """

  def __init__(self, num_units, forget_bias=1.0, input_size=None,
               state_is_tuple=True, activation=tf.tanh):
    """Initialize the basic LSTM cell.
    Args:
      num_units: int, The number of units in the LSTM cell.
      forget_bias: float, The bias added to forget gates (see above).
      input_size: Deprecated and unused.
      state_is_tuple: If True, accepted and returned states are 2-tuples of
        the `c_state` and `m_state`.  If False, they are concatenated
        along the column axis.  The latter behavior will soon be deprecated.
      activation: Activation function of the inner states.
    """
    if not state_is_tuple:
      logging.warn("%s: Using a concatenated state is slower and will soon be "
                   "deprecated.  Use state_is_tuple=True.", self)
    if input_size is not None:
      logging.warn("%s: The input_size parameter is deprecated.", self)
    self._num_units = num_units
    self._forget_bias = forget_bias
    self._state_is_tuple = state_is_tuple
    self._activation = activation

  @property
  def state_size(self):
    return self._num_units

  @property
  def output_size(self):
    return self._num_units

  def __call__(self, inputs, state, scope=None):
        
    with vs.variable_scope(type(self).__name__ + "gru_cell"):  # "BasicLSTMCell"
        with tf.variable_scope("inner_gru_cell"):


          # Parameters of gates are concatenated into one multiply for efficiency.
#           h,_ = array_ops.split(1, 2, state) 
          h = state

    #       concat = self._linear([inputs, h], 2 * self._num_units, True)
          concat = self._linear([inputs, h], 2 * self._num_units, True)


    #         h = (50, 64) and concat = (50, 256)
    #       funsies = h * concat

          r_t, z_t = array_ops.split(1, 2, concat)

          r_t = tf.sigmoid(r_t)
          z_t = tf.sigmoid(z_t)


        with tf.variable_scope("inner_gru_cell_2"):

    #     ValueError: Incompatible shapes for broadcasting: h = (50, 64) and r_t =  (50, 128)
#           funsies = h * r_t

          h_t_ = self._linear([inputs, r_t * h], self._num_units, True)
          h_t_ = tf.tanh(h_t_)
          h_t = (z_t * h) + ((1 - z_t) * h_t_)


    return (h_t, h_t)















  def _get_concat_variable(name, shape, dtype, num_shards):
    """Get a sharded variable concatenated into one tensor."""
    sharded_variable = _get_sharded_variable(name, shape, dtype, num_shards)
    if len(sharded_variable) == 1:
      return sharded_variable[0]
  
    concat_name = name + "/concat"
    concat_full_name = vs.get_variable_scope().name + "/" + concat_name + ":0"
    for value in ops.get_collection(ops.GraphKeys.CONCATENATED_VARIABLES):
      if value.name == concat_full_name:
        return value
  
    concat_variable = array_ops.concat(0, sharded_variable, name=concat_name)
    ops.add_to_collection(ops.GraphKeys.CONCATENATED_VARIABLES,
                          concat_variable)
    return concat_variable
  
  
  def _get_sharded_variable(name, shape, dtype, num_shards):
    """Get a list of sharded variables with the given dtype."""
    if num_shards > shape[0]:
      raise ValueError("Too many shards: shape=%s, num_shards=%d" %
                       (shape, num_shards))
    unit_shard_size = int(math.floor(shape[0] / num_shards))
    remaining_rows = shape[0] - unit_shard_size * num_shards
  
    shards = []
    for i in range(num_shards):
      current_size = unit_shard_size
      if i < remaining_rows:
        current_size += 1
      shards.append(vs.get_variable(name + "_%d" % i, [current_size] +   shape[1:],
                                    dtype=dtype))
    return shards
  
  def _linear(self, args, output_size, bias, bias_start=0.0, scope=None):
    """Linear map: sum_i(args[i] * W[i]), where W[i] is a variable.
    Args:
      args: a 2D Tensor or a list of 2D, batch x n, Tensors.
      output_size: int, second dimension of W[i].
      bias: boolean, whether to add a bias term or not.
      bias_start: starting value to initialize the bias; 0 by default.
      scope: VariableScope for the created subgraph; defaults to "Linear".
    Returns:
      A 2D Tensor with shape [batch x output_size] equal to
      sum_i(args[i] * W[i]), where W[i]s are newly created matrices.
    Raises:
      ValueError: if some of the arguments has unspecified or wrong shape.
    """
    if args is None or (nest.is_sequence(args) and not args):
      raise ValueError("`args` must be specified")
    if not nest.is_sequence(args):
      args = [args]
  
    # Calculate the total size of arguments on dimension 1.
    total_arg_size = 0
    shapes = [a.get_shape().as_list() for a in args]
    for shape in shapes:
      if len(shape) != 2:
        raise ValueError("Linear is expecting 2D arguments: %s" % str(shapes))
      if not shape[1]:
        raise ValueError("Linear expects shape[1] of arguments: %s" % str(  shapes))
      else:
        total_arg_size += shape[1]
  
    dtype = [a.dtype for a in args][0]
  
    # Now the computation.
    with vs.variable_scope(scope or "Linear"):
      matrix = vs.get_variable(
          "Matrix", [total_arg_size, output_size], dtype=dtype)
      if len(args) == 1:
        res = math_ops.matmul(args[0], matrix)
      else:
        res = math_ops.matmul(array_ops.concat(1, args), matrix)
      if not bias:
        return res
      bias_term = vs.get_variable(
          "Bias", [output_size],
          dtype=dtype,
          initializer=init_ops.constant_initializer(
              bias_start, dtype=dtype))
    return res + bias_term

In [ ]:
# ------------------
# YOUR COMPUTATION GRAPH HERE


# create a BasicLSTMCell
#   use it to create a MultiRNNCell
#   use it to create an initial_state
#     note that initial_state will be a *list* of tensors!

# call seq2seq.rnn_decoder

# transform the list of state outputs to a list of logits.
# use a linear transformation.

# call seq2seq.sequence_loss

# create a training op using the Adam optimizer





with tf.variable_scope("lstm_vars") as lstm_vars_scope:


    
    cell = BasicGRUCell( state_dim, forget_bias=1.0)
#     cell = tf.nn.rnn_cell.BasicLSTMCell( state_dim, forget_bias=1.0, state_is_tuple=True)
    multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)

    #  create initial state
    #  put state_is_tuple=True in your MultiRNNcell and this solved mine "Tensor object is not iterate"

    #  takes tuple form (c_state, h_state) for lstm
    initial_state = multi_cell.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.seq2seq.rnn_decoder(inputs, initial_state, multi_cell)


    #  output dim is = at this point, inputs is a list of length sequence_length
    # each element of inputs is [batch_size,vocab_size]
    #  is the output actually batch_size by state_dim?

    #  logits
    W_out = tf.get_variable("w_out", [state_dim,vocab_size], tf.float32,
                            tf.random_normal_initializer())
    b_out = tf.get_variable("b_out", [vocab_size,] , 
                        initializer=tf.constant_initializer(0.1))
    probs = []
    [probs.append(tf.matmul(output, W_out) + b_out) for output in outputs]


    # targets is a list of length sequence_length
    # each element of targets is a 1D vector of length batch_size


    loss_weights = [tf.ones([batch_size]) for i in range(sequence_length)]
    rnn_loss = tf.nn.seq2seq.sequence_loss(probs, targets, loss_weights)
    rnn_optim = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(rnn_loss)











    # ------------------
    # YOUR SAMPLER GRAPH HERE

    # place your sampler graph here it will look a lot like your
    # computation graph, except with a "batch_size" of 1.

    # remember, we want to reuse the parameters of the cell and whatever
    # parameters you used to transform state outputs to logits!

    lstm_vars_scope.reuse_variables()
    
    

    s_cell = BasicGRUCell( state_dim, forget_bias=1.0)
#     s_cell = tf.nn.rnn_cell.BasicLSTMCell( state_dim, forget_bias=1.0, state_is_tuple=True
#                                         )
    s_multi_cell = tf.nn.rnn_cell.MultiRNNCell([s_cell] * num_layers, state_is_tuple=True
                                        )

    #  create initial state
    #  put state_is_tuple=True in your MultiRNNcell and this solved mine "Tensor object is not iterate"

    #  takes tuple form (c_state, h_state) for lstm
    s_initial_state = multi_cell.zero_state(sampler_batch_size, tf.float32)
    s_outputs, s_final_state = tf.nn.seq2seq.rnn_decoder(s_inputs, s_initial_state, s_multi_cell)


    #  output dim is = at this point, inputs is a list of length sequence_length
    # each element of inputs is [batch_size,vocab_size]
    #  is the output actually batch_size by state_dim?

    #  logits
    s_W_out = tf.get_variable("w_out", [state_dim,vocab_size], tf.float32,
                            tf.random_normal_initializer())
    s_b_out = tf.get_variable("b_out", [vocab_size,] , 
                        initializer=tf.constant_initializer(0.1))
    s_probs = []
    [s_probs.append(tf.matmul(output, s_W_out) + s_b_out) for output in s_outputs]


    # targets is a list of length sequence_length
    # each element of targets is a 1D vector of length batch_size


#     s_loss_weights = [tf.ones([sampler_batch_size]) for i in range(sampler_seq_len)]
#     s_rnn_loss = tf.nn.seq2seq.sequence_loss(s_probs, targets, s_loss_weights)
#     s_rnn_optim = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(s_rnn_loss)










#
# ==================================================================
# ==================================================================
# ==================================================================
#

def sample( num=200, prime='ab' ):

    # prime the pump 

    # generate an initial state. this will be a list of states, one for
    # each layer in the multicell.
    s_state = sess.run( s_initial_state )

    # for each character, feed it into the sampler graph and
    # update the state.
    for char in prime[:-1]:
        x = np.ravel( data_loader.vocab[char] ).astype('int32')
        feed = { s_inputs_ph:x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        s_state = sess.run( s_final_state, feed_dict=feed )

    # now we have a primed state vector; we need to start sampling.
    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.ravel( data_loader.vocab[char] ).astype('int32')

        # plug the most recent character in...
        feed = { s_inputs_ph:x }
        for i, s in enumerate( s_initial_state ):
            feed[s] = s_state[i]
        ops = [s_probs]
        ops.extend( list(s_final_state) )

        retval = sess.run( ops, feed_dict=feed )

        s_probsv = retval[0]
        s_state = retval[1:]

        # ...and get a vector of probabilities out!

        # now sample (or pick the argmax)
        sample = np.argmax( s_probsv[0][0] )
#         sample = np.random.choice( vocab_size, p=s_probsv[0][0] )

        pred = data_loader.chars[sample]
        ret += pred
        char = pred

    return ret

#
# ==================================================================
# ==================================================================
# ==================================================================
#

sess = tf.Session()
sess.run( tf.initialize_all_variables() )
summary_writer = tf.train.SummaryWriter( "./tf_logs", graph=sess.graph )

lts = []

print ("FOUND %d BATCHES" % data_loader.num_batches)

for j in range(100):

    state = sess.run( initial_state )
    data_loader.reset_batch_pointer()

    for i in range( data_loader.num_batches ):
        
        x,y = data_loader.next_batch()

        # we have to feed in the individual states of the MultiRNN cell
        feed = { in_ph: x, targ_ph: y }
        for k, s in enumerate( initial_state ):
            feed[s] = state[k]

        ops = [rnn_optim,rnn_loss]
        ops.extend( list(final_state) )

        # retval will have at least 3 entries:
        # 0 is None (triggered by the optim op)
        # 1 is the loss
        # 2+ are the new final states of the MultiRNN cell
        retval = sess.run( ops, feed_dict=feed )

        lt = retval[1]
        state = retval[2:]

        if i%1000==0:
            print ("%d %d\t%.4f" % ( j, i, lt ))
            lts.append( lt )

    print (sample( num=60, prime="And " ))
#     print (sample( num=60, prime="ababab" ))
#     print (sample( num=60, prime="foo ba" ))
#     print (sample( num=60, prime="abcdab" ))

summary_writer.close()

#
# ==================================================================
# ==================================================================
# ==================================================================
#



#import matplotlib
#import matplotlib.pyplot as plt
#plt.plot( lts )
#plt.show()

# Results

## LSTM Alma.txt

FOUND 186 BATCHES
0 0	4.2935
And EElll  topppppiiittot tttee ttrre ttre ttre tthe the tr the 
1 0	3.4265
And ttEe oll to the the ann the ant he and the the the the the t
2 0	3.0400
And tthe essan to the and the and the the the the the the the th
3 0	2.8064
And tthe esan to the and the and the the the the the the the the
4 0	2.6638
And tthe esan to the and the the the the the the the the the the
5 0	2.5581
And dte son to the and the the pore to the the the the the the t
6 0	2.4763
And mman to the ass and the the wor the the wor the the wor the 
7 0	2.4090
And mman to the ass and the the word the the word the the word t
8 0	2.3508
And mman to the ass and the the word the the word the the word t
9 0	2.2986
And mman to the ass and the the word the the word the the word t
10 0	2.2507
And mman tho the ward the the word the the word the the word the
11 0	2.2062
And mman tho the ward the the word the were the the word the wer
12 0	2.1643
And mman tho has ward the were the the word the were the

## GRU Alma.txt

FOUND 186 BATCHES
0 0	4.2517
And the and the the the the the the the the the the the the the 
1 0	2.5859
And the the the the the the the the the the the the the the the 
2 0	2.3371
And the the the the the the were the the were the the wir the we
3 0	2.2042
And the the the the were the the were the were the were the were
4 0	2.1065
And the the the were the were the were the were the reand the re
5 0	2.0263
And the the Lamanit the Lamanit the Lamanit the Lamanit the Lama
6 0	1.9556
And the the Lamanit the Lamanit the Lamanit the Lamanit the Lama
7 0	1.8889
And the the Lamanite the Lamanites of the Lamanites of the Laman
8 0	1.8244
And the will des the Lamanites of the Lamanites of the Lamanites
9 0	1.7635
And the will de with the Lamanites and the Lamanites and the Lam
10 0	1.7089
And the will de with the Lamanites and the Lamanites and the Lam
11 0	1.6615
And the will de with the Lamanites and the Lamanites and the Lam
12 0	1.6203
And the will de with the Lamanites and the Lamanites and

## GRU Lil Shakespear

FOUND 446 BATCHES
0 0	4.2259
And myo mand she mand sher and sound sour the sour the sour the 
1 0	2.5050
And ghat hand she mand she mand she sher and and and the sour th
2 0	2.3541
And ghat he mand and and and the seand the sear the sear the sea
3 0	2.2653
And ghat hand and the sour the sear the sour the sour the sour t
4 0	2.1954
And ghat hand mand mand the sour the sour the sour the sour the 
5 0	2.1508
And ghat hand
What he shall the sore the sore the sore the sore 
6 0	2.1252
And greave
The sore the sore the sore the sore the sore the sore
7 0	2.1065
And greave
The sore the sore the sone the sone the sone the sone
8 0	2.0891
And greave
The sare the sone the sone the sone the sone the sone
9 0	2.0712
And greather
And the sone the sone the sone the sone the sone th
10 0	2.0536
And great
The seep the seep the seep the sone the sone the sone 
11 0	2.0364
And great
The seep the seep the seep the sone the sone the sone

12 0	2.0193
And great
The seep the seep the seep the sone the sone t

KeyboardInterrupt: 

## GRU Test1.txt

FOUND 11 BATCHES
0 0	2.3998
foo bafo                                                          
1 0	1.9070
foo baaf o    o       o        o        o        o        o       
2 0	1.5982
foo bar foo ba  foo    fo    o    o    o    o    o    o    o    o 
3 0	1.3310
foo bar foo bar foo ba  foo  ba  foo  ba  foo  ba  foo  ba  foo  b
4 0	1.0850
foo bar foo bar foo bar foo bar foo bar foo bar foo bar foo bar fo
5 0	0.8619
foo bar foo bar foo bar foo bar foo bar foo bar foo bar foo bar fo
6 0	0.6602
foo barrfoo bar foo bar foo bar foo bar foo bar foo bar foo bar fo
7 0	0.4658
foo barrfooobarrfooobarrfooobarrfooobarrfooobarrfooobarrfooobarrfo
8 0	0.4487
foo barrfooobaar foo bar foo bar foo bar foo bar foo bar foo bar f
9 0	0.2904
foo barrfooobaar foo bar foo bar foo bar foo bar foo bar foo bar f
10 0	0.2603
foo barrfooobaar foo bar foo bar foo bar foo bar foo bar foo bar f
11 0	0.2297
foo barr foo bar foo bar foo bar foo bar foo bar foo bar foo bar f
12 0	0.2085
foo barr foo bar foo bar foo bar 

## GRU Abcd

FOUND 6 BATCHES
0 0	1.3971
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
1 0	1.2216
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
2 0	1.0775
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
3 0	0.9380
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
4 0	0.8016
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
5 0	0.6685
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
6 0	0.5405
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
7 0	0.4201
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
8 0	0.3098
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
9 0	0.2109
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
10 0	0.1246
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
11 0	0.0639
abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab
12 0	0.0465
abcdabcdabcdabcdabcdabcdabcdabcdab

## GRU Abab

FOUND 13 BATCHES
0 0	0.7311
ababababababababababababababababababababababababababababababababab
1 0	0.5771
ababababababababababababababababababababababababababababababababab
2 0	0.4248
ababababababababababababababababababababababababababababababababab
3 0	0.2599
ababababababababababababababababababababababababababababababababab
4 0	0.0943
ababababababababababababababababababababababababababababababababab
5 0	0.0189
ababababababababababababababababababababababababababababababababab
6 0	0.0164
ababababababababababababababababababababababababababababababababab
7 0	0.0154
ababababababababababababababababababababababababababababababababab
8 0	0.0148
ababababababababababababababababababababababababababababababababab
9 0	0.0143
ababababababababababababababababababababababababababababababababab
10 0	0.0138
ababababababababababababababababababababababababababababababababab
11 0	0.0134
ababababababababababababababababababababababababababababababababab
12 0	0.0130
ababababababababababababababababa

KeyboardInterrupt: 

## GRU Romeo And Juliet

FOUND 55 BATCHES
0 0	4.2070
And aro t ee ee te e ee e ee e ee e ee e ee e ee e ee e ee e ee 
1 0	3.2948
And athe the the the the the the the the the the the the the the
2 0	3.1015
And bor the an the the the the the the the the the the the the t
3 0	2.8894
And bour the the sand the the the the the the the the the the th
4 0	2.7518
And ber the the se and the the the the the the the the the the t
5 0	2.6691
And dand the the se the the the the the the the the the the the 
6 0	2.6108
And dand the the she se the the the the the the the the the the 
7 0	2.5643
And dand the the she the the the the the the the the the the the
8 0	2.5256
And dand the the she the the the the the the the the the the the
9 0	2.4928
And dand the the she the seand the the the the the the the the t
10 0	2.4646
And dand the the she the seand the the the the the the the the t
11 0	2.4399
And dour the the she the seand with the the seand with the the s
12 0	2.4179
And dour the the she the seand with the the seand with th